In [1]:
# This script is for making sure that the average number of cells sampled across a variance level converges to the expected baseline proportion

In [21]:
import os
import pandas as pd
from functools import reduce
import re
import json
import plotly.express as px

In [22]:
path = "/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/testrun/simulations/0/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv"
proportions_json_path= "/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/cell_type_proportions.json"

In [23]:
def getBaselineProportion(CTProfile_name:str, proportions_json_path:json) -> dict:
    """Get the list of proper cell type baseline proportions depending on if we are using brain or pbmc

    Args:
        CTProfile_name (str): name of the dataframe of cell type profiles
        proportions_json (json): json that contains baseline cell type profiles for brian or pbmc

    Returns:
        dict: A dict where keys are cell types, and values are their expected baseline proportions
    """
    
    # open json file
    with open(proportions_json_path, 'r') as json_file:
        dict_propostions_json = json.load(json_file)
    
        print(dict_propostions_json)
    
    # Get the correct key
    for key in dict_propostions_json.keys():
        if key == CTProfile_name:
            correct_key = key
    # If no key was found,raise error
    if not correct_key:
        raise ValueError("Key not found")
    
    # Return the proportions of cell types based on our key (which is a tissue type)
    return dict_propostions_json[correct_key]
baseline_proportion = getBaselineProportion(CTProfile_name="brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv", proportions_json_path=proportions_json_path)

    

{'pbmc_sc_with_metadata_cpm_pc_cell_type_profiles.csv': {'B-cells': 0.05, 'Dendritic cells': 0.05, 'Macrophages': 0.2, 'Monocytes': 0.05, 'Nk-cells': 0.1, 'Platelets': 0.05, 'T-cells': 0.5}, 'brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv': {'Astrocytes': 0.05, 'Excitatory neurons': 0.5, 'Inhibitory neurons': 0.3, 'Microglial cells': 0.05, 'Oligodendrocyte precursor cells': 0.05, 'Oligodendrocytes': 0.05}}


In [24]:
#### Get all the simulation files that contain how many cells were subasampled for each simulation

# Get a list of all files in the directory
all_files = os.listdir(path)

pattern = ".*n_sim.*"

# Initialize a list to store matching file names
matching_files = []

# Loop through the files and check if they match the pattern
for file_name in all_files:
    if re.match(pattern, file_name):
        matching_files.append(file_name)
        
        
        
matching_files




FileNotFoundError: [Errno 2] No such file or directory: '/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/testrun/simulations/0/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv'

In [25]:
def getMatchingFiles(path):
    
    lofiles = os.listdir(path)


    pattern = ".*n_sim.*"

    # Initialize a list to store matching file names
    matching_files = []

    # Loop through the files and check if they match the pattern
    for file_name in lofiles:
        if re.match(pattern, file_name):
            matching_files.append(f"{path}/{file_name}")
        
    return matching_files
        

In [26]:
lopaths = [os.path.join(path, file) for file in matching_files]
lopaths

NameError: name 'matching_files' is not defined

In [27]:
lodfs = [pd.read_csv(file_path, index_col=0) for file_path in lopaths]
lodfs

NameError: name 'lopaths' is not defined

In [28]:
merged_df = pd.concat(lodfs, axis = 1)
merged_df = merged_df.sum(axis = 1)
merged_df

NameError: name 'lodfs' is not defined

In [29]:
def normalize_series_by_sum(series):
    """
    Normalize a pandas Series by dividing each value by the sum of the Series.

    Args:
        series (pandas.Series): The Series to normalize.

    Returns:
        pandas.Series: A new Series with values normalized by the sum.
    """
    # Calculate the sum of the Series
    series_sum = series.sum()

    # Normalize the Series by dividing each value by the sum
    normalized_series = series / series_sum

    return normalized_series

def getCellAverage(loFiles):
    lodfs = [pd.read_csv(file_path, index_col=0) for file_path in loFiles]
    
    merged_df = pd.concat(lodfs, axis = 1)
    merged_df = merged_df.sum(axis = 1)

    normalized_series = normalize_series_by_sum(merged_df)
    
    return normalized_series


In [30]:
retrieved_proportion = normalize_series_by_sum(merged_df)
retrieved_proportion

,0.005,0.75,0.02,0.08,0.085,10,0,0.05,0.075,0.01,...,0.03,0.1,0.045,0.35,0.5,0.07,1000,0.2,0.45,0.035
Astrocytes,0.050090,0.078115,0.049280,0.051005,0.052960,0.163970,0.05,0.049020,0.050920,0.050105,...,0.049330,0.052095,0.049070,0.074450,0.074260,0.050530,0.120805,0.059860,0.070975,0.049545
Excitatory neurons,0.499530,0.365305,0.501060,0.499065,0.501355,0.216805,0.50,0.502035,0.504790,0.500065,...,0.500335,0.499655,0.500405,0.443215,0.443590,0.502315,0.234160,0.488970,0.447270,0.501630
Inhibitory neurons,0.300205,0.302520,0.299640,0.296795,0.297135,0.260490,0.30,0.300330,0.295820,0.299545,...,0.301800,0.295860,0.299250,0.273140,0.254245,0.295135,0.202030,0.295195,0.267070,0.298585
Microglial cells,0.050155,0.081205,0.050060,0.048445,0.048975,0.120270,0.05,0.048715,0.048590,0.050060,...,0.050490,0.048750,0.049865,0.068165,0.060325,0.050565,0.149330,0.051970,0.076005,0.051105
Oligodendrocyte precursor cells,0.049945,0.095185,0.050695,0.051935,0.050310,0.122705,0.05,0.050575,0.048265,0.050175,...,0.049020,0.054630,0.052375,0.068495,0.095330,0.051225,0.138595,0.055165,0.076085,0.050130
Oligodendrocytes,0.050075,0.077670,0.049265,0.052755,0.049265,0.115760,0.05,0.049325,0.051615,0.050050,...,0.049025,0.049010,0.049035,0.072535,0.072250,0.050230,0.155080,0.048840,0.062595,0.049005


In [31]:
variance_int = path.split("/")[-2]
variance_int

'0'

In [32]:
#os.mkdir("../../data/retrieved_proportions")
#os.mkdir("../../data/retrieved_proportions/brain")
retrieved_proportion.to_csv(f"../../data/retrieved_proportions/brain/var{variance_int}.csv")

In [40]:
def main():
    main_path = "/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations"
    
    lovariances = os.listdir(main_path)
    
    lopaths = [f"{main_path}/{variance}/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv" for variance in lovariances]
    print(lopaths)
    
    
    lolofiles = [getMatchingFiles(path) for path in lopaths]
    print(lolofiles)
    
    dict_variance_average = {}
    
    
    for i, loFiles in enumerate(lolofiles):
        cell_averages = getCellAverage(loFiles)

        dict_variance_average[lovariances[i]] = cell_averages
        
    #print(dict_variance_average)
    return dict_variance_average
    
dict_variance_average = main()

['/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations/0.005/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv', '/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations/0.75/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv', '/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations/0.02/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv', '/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations/0.08/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv', '/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/bulkSimulationOneProfile/data/1004_in_numerator_sqrtvariance/simulations/0.085/brain_sc_with_metadata_cpm_pc_cell_type_profiles.csv', '/space/grp/aadrian/Pseudob

In [41]:
merged_df = pd.DataFrame(dict_variance_average)
merged_df

,0.005,0.75,0.02,0.08,0.085,10,0,0.05,0.075,0.01,...,0.03,0.1,0.045,0.35,0.5,0.07,1000,0.2,0.45,0.035
Astrocytes,0.050090,0.078115,0.049280,0.051005,0.052960,0.163970,0.05,0.049020,0.050920,0.050105,...,0.049330,0.052095,0.049070,0.074450,0.074260,0.050530,0.120805,0.059860,0.070975,0.049545
Excitatory neurons,0.499530,0.365305,0.501060,0.499065,0.501355,0.216805,0.50,0.502035,0.504790,0.500065,...,0.500335,0.499655,0.500405,0.443215,0.443590,0.502315,0.234160,0.488970,0.447270,0.501630
Inhibitory neurons,0.300205,0.302520,0.299640,0.296795,0.297135,0.260490,0.30,0.300330,0.295820,0.299545,...,0.301800,0.295860,0.299250,0.273140,0.254245,0.295135,0.202030,0.295195,0.267070,0.298585
Microglial cells,0.050155,0.081205,0.050060,0.048445,0.048975,0.120270,0.05,0.048715,0.048590,0.050060,...,0.050490,0.048750,0.049865,0.068165,0.060325,0.050565,0.149330,0.051970,0.076005,0.051105
Oligodendrocyte precursor cells,0.049945,0.095185,0.050695,0.051935,0.050310,0.122705,0.05,0.050575,0.048265,0.050175,...,0.049020,0.054630,0.052375,0.068495,0.095330,0.051225,0.138595,0.055165,0.076085,0.050130
Oligodendrocytes,0.050075,0.077670,0.049265,0.052755,0.049265,0.115760,0.05,0.049325,0.051615,0.050050,...,0.049025,0.049010,0.049035,0.072535,0.072250,0.050230,0.155080,0.048840,0.062595,0.049005


In [42]:
melted = merged_df.reset_index().melt(id_vars=['index'])
melted = melted.sort_values("variable")
melted


,index,variable,value
36,Astrocytes,0,0.050000
37,Excitatory neurons,0,0.500000
38,Inhibitory neurons,0,0.300000
41,Oligodendrocytes,0,0.050000
40,Oligodendrocyte precursor cells,0,0.050000
...,...,...,...
103,Excitatory neurons,10000,0.259390
105,Microglial cells,10000,0.153865
106,Oligodendrocyte precursor cells,10000,0.160465
107,Oligodendrocytes,10000,0.108785


In [43]:
fig = px.line(melted, x = 'variable', y = 'value', color = 'index')

fig.update_layout(title = "Average Percent Composition of CTs Across Variance Levels")

# Update x-axis and y-axis labels
fig.update_xaxes(title_text='Variance')
fig.update_yaxes(title_text='Average Percent Composition')
fig.update_layout(legend_title_text='Cell Types (CTs)')

In [18]:
merged_df

,0.07,0.35,0.065,10,10000,1000,0.095,0.45,0.015,0.04,...,0.06,0.02,0.75,0.08,0.025,0.2,1,0.5,0.05,0.055
Astrocytes,0.073700,0.152780,0.096840,0.176760,0.166445,0.206030,0.093775,0.189370,0.056335,0.071690,...,0.09429,0.059885,0.153155,0.115065,0.061390,0.165680,0.188800,0.160290,0.08221,0.068580
Excitatory neurons,0.406930,0.188085,0.379615,0.089845,0.095245,0.116390,0.352960,0.179415,0.490765,0.439055,...,0.39265,0.471145,0.132975,0.369320,0.473685,0.272670,0.149410,0.183240,0.41874,0.410585
Inhibitory neurons,0.237460,0.161440,0.238065,0.152255,0.099670,0.126440,0.206215,0.151595,0.294075,0.249205,...,0.22733,0.285760,0.135025,0.206255,0.282665,0.145190,0.115665,0.124815,0.24400,0.238510
Microglial cells,0.080105,0.168760,0.081265,0.263675,0.161920,0.174455,0.117300,0.182995,0.054855,0.076060,...,0.09575,0.057745,0.167070,0.110965,0.065160,0.135215,0.194635,0.204610,0.07255,0.094080
Oligodendrocyte precursor cells,0.096720,0.162445,0.099210,0.164160,0.251700,0.214305,0.104060,0.133015,0.051885,0.070635,...,0.09285,0.066915,0.217545,0.124660,0.066080,0.166730,0.133495,0.180255,0.08811,0.114170
Oligodendrocytes,0.105085,0.166490,0.105005,0.153305,0.225020,0.162380,0.125690,0.163610,0.052085,0.093355,...,0.09713,0.058550,0.194230,0.073735,0.051020,0.114515,0.217995,0.146790,0.09439,0.074075
